In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/보유리스크율_일반/20210728')
BASE_YYMM = '202012'
BASE_YYYY = BASE_YYMM[:4]

In [3]:
# 데이터 불러오기
일반_원수_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간경과보험료_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'OGL_ELP_PRM': float})
일반_출재_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간경과보험료_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RN_ELP_PRM': float})
일반_원수_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간손해액_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'OGL_LOSS': float})
일반_출재_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간손해액_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'RN_LOSS': float})
일반_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_ELP_PRM_1YR': float, 'RN_ELP_PRM_1YR': float})
일반_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_LOSS_1YR': float, 'RN_LOSS_1YR': float})
일반_특약보종별_경과보험료 = pd.read_excel(FILE_PATH / f'일반_특약보종별_경과보험료_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_ELP_PRM': float, 'RN_ELP_PRM': float})
일반_특약보종별_손해액 = pd.read_excel(FILE_PATH / f'일반_특약보종별_손해액_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_LOSS': float, 'RN_LOSS': float})
일반_특약수수료 = pd.read_excel(FILE_PATH / f'일반_특약수수료_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_특약정보 = pd.read_excel(FILE_PATH / f'일반_특약정보.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
산업_손해율_변동계수 = pd.read_excel(FILE_PATH / f'산업_손해율_변동계수_{BASE_YYMM}.xlsx')
일반_직전1년마감실적 = pd.read_excel(FILE_PATH / f'일반_연마감실적_{BASE_YYMM}.xlsx', dtype={'보험종목': str})
일반_1년전1년마감실적 = pd.read_excel(FILE_PATH / f'일반_연마감실적_{int(BASE_YYMM)-100}.xlsx', dtype={'보험종목': str})
일반_2년전1년마감실적 = pd.read_excel(FILE_PATH / f'일반_연마감실적_{int(BASE_YYMM)-200}.xlsx', dtype={'보험종목': str})
일반_위험계수 = pd.read_excel(FILE_PATH / f'일반_위험계수_{BASE_YYMM}.xlsx')

In [4]:
pd.options.display.float_format = '{:,.3f}'.format

# 위험계수
def get_comb_ratio(data: pd.DataFrame) -> pd.DataFrame:
    boz_cd_mapper = {'근재': 'A005', '기술':  'A002', '기타':  'A010', '농작물재해보상':  'A009', '상해':  'A007', '외국인상해':  'A008', '종합':  'A003', '책임':  'A006', '해상':  'A004', '화재':  'A001'}
    return data \
        .loc[lambda x: x['당월/누계'] == '누계'] \
        .query('대상구분 == "보유"') \
        .query('~중분류.str.contains("해외")', engine='python') \
        .query('~중분류.str.contains("보증")', engine='python') \
        .assign(중분류 = lambda x: x.중분류.str.replace(r'\d+(.) ', '', regex=True)) \
        .assign(중분류 = lambda x: x.중분류.str.replace('권원보험', '기타특종')) \
        .assign(중분류 = lambda x: x.중분류.str.replace('기타특종', '기타')) \
        .assign(중분류 = lambda x: x.중분류.str.replace('근로자재해보상', '근재')) \
        .assign(중분류 = lambda x: x.중분류.str.replace('종합보험', '종합')) \
        .assign(KICS분류 = lambda x: np.where(x.보험종목.isin(['10902', '10903']), '농작물재해보상', np.where(x.보험종목 == '10607', '외국인상해', x.중분류))) \
        .assign(BOZ_CD = lambda x: x['KICS분류'].apply(lambda y: boz_cd_mapper.get(y, '#'))) \
        .groupby('BOZ_CD')[['경과보험료', 'IFRS손해액', '순사업비']].sum() \
        .eval('합산비율 = (IFRS손해액+순사업비)/경과보험료')[['합산비율']]

일반_직전1년마감실적_가공 = get_comb_ratio(일반_직전1년마감실적) \
    .rename(columns={'합산비율': '합산비율_직전1년'}) \
    .reset_index()

일반_1년전1년마감실적_가공 = get_comb_ratio(일반_1년전1년마감실적) \
    .rename(columns={'합산비율': '합산비율_1년전1년'}) \
    .reset_index()

일반_2년전1년마감실적_가공 = get_comb_ratio(일반_2년전1년마감실적) \
    .rename(columns={'합산비율': '합산비율_2년전1년'}) \
    .reset_index()

일반_위험계수_가공 = 일반_2년전1년마감실적_가공 \
    .merge(일반_1년전1년마감실적_가공, on='BOZ_CD', how='outer') \
    .merge(일반_직전1년마감실적_가공, on='BOZ_CD', how='outer') \
    .merge(일반_위험계수, on='BOZ_CD', how='outer') \
    .eval('COMB_RATIO = (합산비율_직전1년 + 합산비율_1년전1년 + 합산비율_2년전1년)/3') \
    .assign(RSK_COEF_PRM = lambda x: np.fmax(x.BSE_RSK_COEF_PRM*0.7, x.BSE_RSK_COEF_PRM+(x.COMB_RATIO-x.BSE_COMB_RATIO)*0.5)) \
    [['BOZ_CD', 'COMB_RATIO', 'RSK_COEF_PRM', 'RSK_COEF_RSV']]

In [8]:
일반_특약정보_가공.loc[일반_특약정보_가공['TTY_CD_GRP'].isna()]

,RRNR_TTY_CD,TTY_CD_NM,TTY_YR,TTY_CD_GRP
44,U2007LGQ1050001000,가축보험 비례재보험,2007,None
45,U2008KRQ1040001000,풍수해보험보통약관1,2008,None
46,U2008KRQ1040002000,풍수해보험보통약관2,2008,None
52,U2009KRQ1040001000,풍수해보험보통약관1&3,2009,None
53,U2009KRQ1040002000,풍수해보험보통약관2,2009,None
64,U1999KRS1040001001,화재보험 제1차 초과액특약1999 KOREAN RE,1999,None
65,U1999KRS1040001002,화재보험 제2차 초과액특약1999 KOREAN RE,1999,None
66,U1999KRS1040001004,재산종합보험재보험및재재보험특약1999KOREAN Re,1999,None
67,U2010KRQ1040001000,풍수해보험보통약관1&3,2010,None
68,U2010KRQ1040002000,풍수해보험보통약관2,2010,None


In [9]:
pd.options.display.float_format = '{:,.3f}'.format

# 특약정보
## 근재보험특약, 기술보험특약, 재물보험특약, 배상책임보험특약, 해외PST
일반_특약정보_가공 = 일반_특약정보.copy()
일반_특약정보_가공['TTY_CD_GRP'] = None
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("(재물보험|패키지)"), 'TTY_CD_GRP'] = '재물보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("기술보험"), 'TTY_CD_GRP'] = '기술보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("근재보험"), 'TTY_CD_GRP'] = '근재보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("배상책임보험"), 'TTY_CD_GRP'] = '배상책임보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("해외PST"), 'TTY_CD_GRP'] = '해외PST'
일반_특약정보_가공.loc[lambda x: x.RRNR_TTY_CD == "U2020OVQ1058002000", 'TTY_CD_GRP'] = '상해보험특약'


# 특약수수료
일반_특약수수료_가공 = 일반_특약수수료.copy()
일반_특약수수료_가공 = 일반_특약수수료_가공.merge(일반_특약정보_가공, on=['RRNR_TTY_CD', 'TTY_YR'], how='left').query('~TTY_CD_GRP.isna()')
일반_특약수수료_가공 = 일반_특약수수료_가공[['TTY_CD_GRP', 'TTY_YR', 'CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'LWT_CMSN_RT', 'TOP_CMSN_RT']].drop_duplicates()


# 직전3년평균손해율
일반_원수_직전3년연간경과보험료_가공 = 일반_원수_직전3년연간경과보험료.copy()
일반_원수_직전3년연간경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간경과보험료_가공, 일반_상품정보)
일반_원수_직전3년연간경과보험료_집계 = 일반_원수_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM']].sum()

일반_출재_직전3년연간경과보험료_가공 = 일반_출재_직전3년연간경과보험료.copy()
일반_출재_직전3년연간경과보험료_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간경과보험료_가공)
일반_출재_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간경과보험료_가공, 일반_상품정보)
일반_출재_직전3년연간경과보험료_집계 = 일반_출재_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_ELP_PRM']].sum()

일반_원수_직전3년연간손해액_가공 = 일반_원수_직전3년연간손해액.copy()
일반_원수_직전3년연간손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간손해액_가공, 일반_상품정보)
일반_원수_직전3년연간손해액_집계 = 일반_원수_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_LOSS']].sum()

일반_출재_직전3년연간손해액_가공 = 일반_출재_직전3년연간손해액.copy()
일반_출재_직전3년연간손해액_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간손해액_가공)
일반_출재_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간손해액_가공, 일반_상품정보)
일반_출재_직전3년연간손해액_집계 = 일반_출재_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_LOSS']].sum()

일반_직전3년평균손해율 = 일반_원수_직전3년연간경과보험료_집계 \
    .merge(일반_출재_직전3년연간경과보험료_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_원수_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_출재_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer')
일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)
일반_직전3년평균손해율.eval('RET_ELP_PRM = OGL_ELP_PRM - RN_ELP_PRM', inplace=True)
일반_직전3년평균손해율.eval('RET_LOSS = OGL_LOSS - RN_LOSS', inplace=True)
일반_직전3년평균손해율.eval('LOSS_RATIO = RET_LOSS/RET_ELP_PRM', inplace=True)
일반_직전3년평균손해율 = 일반_직전3년평균손해율.groupby('BOZ_CD', as_index=False)['LOSS_RATIO'].mean()


# 특약보종별 직전1년실적
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료.copy()
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년경과보험료_가공, 일반_상품정보)
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()

일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액.copy()
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년손해액_가공, 일반_상품정보)
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_LOSS_1YR', 'RN_LOSS_1YR']].sum()

일반_특약보종별_직전1년실적 = 일반_특약보종별_직전1년경과보험료_가공 \
    .merge(일반_특약보종별_직전1년손해액_가공, on=['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], how='outer')
일반_특약보종별_직전1년실적[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR', 'OGL_LOSS_1YR', 'RN_LOSS_1YR']] = 일반_특약보종별_직전1년실적[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR', 'OGL_LOSS_1YR', 'RN_LOSS_1YR']].fillna(0)

# 일반_특약보종별_실적_가공 = []
# for tty_cd_grp in ['재물보험특약', '기술보험특약', '근재보험특약', '배상책임보험특약', '해외PST']:
#     STRT_YYYY = str(int(BASE_YYYY)  - (7 if tty_cd_grp in ['근재보험특약', '배상책임보험특약'] else 5))
#     일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR > @STRT_YYYY'))
# 일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)
일반_특약보종별_직전1년실적_가공 = []
for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
    STRT_YYYY = str(int(BASE_YYYY)  - (7 if boz_cd in ['A005', 'A006'] else 5))
    일반_특약보종별_직전1년실적_가공.append(일반_특약보종별_직전1년실적.query('BOZ_CD == @boz_cd').query('TTY_YR > @STRT_YYYY'))
일반_특약보종별_직전1년실적_가공 = pd.concat(일반_특약보종별_직전1년실적_가공, axis=0).reset_index(drop=True)


# 특약보종별 실적
일반_특약보종별_경과보험료_가공 = 일반_특약보종별_경과보험료.copy()
일반_특약보종별_경과보험료_가공 = 일반_특약보종별_경과보험료_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_경과보험료_가공)
일반_특약보종별_경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_경과보험료_가공)
일반_특약보종별_경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_경과보험료_가공, 일반_상품정보)
일반_특약보종별_경과보험료_가공 = 일반_특약보종별_경과보험료_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_경과보험료_가공 = 일반_특약보종별_경과보험료_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()

일반_특약보종별_손해액_가공 = 일반_특약보종별_손해액.copy()
일반_특약보종별_손해액_가공 = 일반_특약보종별_손해액_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_손해액_가공)
일반_특약보종별_손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_손해액_가공)
일반_특약보종별_손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_손해액_가공, 일반_상품정보)
일반_특약보종별_손해액_가공 = 일반_특약보종별_손해액_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_손해액_가공 = 일반_특약보종별_손해액_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_LOSS', 'RN_LOSS']].sum()

일반_특약보종별_실적 = 일반_특약보종별_경과보험료_가공 \
    .merge(일반_특약보종별_손해액_가공, on=['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], how='outer')
일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)

# 일반_특약보종별_실적_가공 = []
# for tty_cd_grp in ['재물보험특약', '기술보험특약', '근재보험특약', '배상책임보험특약', '해외PST']:
#     STRT_YYYY = str(int(BASE_YYYY)  - (7 if tty_cd_grp in ['근재보험특약', '배상책임보험특약'] else 5))
#     일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR > @STRT_YYYY'))
# 일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)
일반_특약보종별_실적_가공 = []
for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
    STRT_YYYY = str(int(BASE_YYYY)  - (7 if boz_cd in ['A005', 'A006'] else 5))
    일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('BOZ_CD == @boz_cd').query('TTY_YR > @STRT_YYYY'))
일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)

# 특약보종별 직전1년실적, 실적 집계
일반_특약보종별_실적_집계 = 일반_특약보종별_실적_가공 \
    .merge(일반_특약보종별_직전1년실적_가공, on=['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], how='outer')

C:\Users\11700205.EDBINS\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [10]:
pd.options.display.float_format = '{:,.0f}'.format

# 보유리스크율
## 위험계수적용법
일반_보유리스크율_위험계수적용법 = []
for tty_cd_grp in 일반_특약보종별_직전1년실적_가공['TTY_CD_GRP'].unique():
    for boz_cd in 일반_특약보종별_직전1년실적_가공['BOZ_CD'].unique():
        prem = 일반_특약보종별_직전1년실적_가공.query('TTY_CD_GRP == @tty_cd_grp').query('BOZ_CD == @boz_cd').groupby('TTY_YR', as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()
        if len(prem) == 0: continue
        comm = 일반_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').drop('TTY_CD_GRP', axis=1)
        loss_ratio = 일반_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        rsk_coef = 일반_위험계수_가공.query('BOZ_CD == @boz_cd')['RSK_COEF_PRM'].values[0]
        일반_보유리스크율_위험계수적용법_특약보종별 = get_ret_risk_rate_by_risk_coef(boz_cd, tty_cd_grp, comm, prem, loss_ratio, rsk_coef)
        일반_보유리스크율_위험계수적용법.append(일반_보유리스크율_위험계수적용법_특약보종별)
일반_보유리스크율_위험계수적용법 = pd.concat(일반_보유리스크율_위험계수적용법, axis=0)
일반_보유리스크율_위험계수적용법.query('OGL_ELP_PRM_1YR > 0', inplace=True)

## 손해율분포법
일반_보유리스크율_손해율분포법 = []
for tty_cd_grp in 일반_특약보종별_실적_집계['TTY_CD_GRP'].unique():
    for boz_cd in 일반_특약보종별_실적_집계['BOZ_CD'].unique():
        for tty_yr in 일반_특약보종별_실적_집계['TTY_YR'].unique():
            ogl_elp_prm_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['OGL_ELP_PRM'].sum()
            rn_elp_prm_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['RN_ELP_PRM'].sum()
            ogl_loss_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['OGL_LOSS'].sum()
            rn_loss_tty = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')['RN_LOSS'].sum()
            ogl_1yr_elp_prm_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['OGL_ELP_PRM_1YR'].sum()
            rn_1yr_elp_prm_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['RN_ELP_PRM_1YR'].sum()
            ogl_1yr_loss_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['OGL_LOSS_1YR'].sum()
            rn_1yr_loss_tty_boz = 일반_특약보종별_실적_집계.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr').query('BOZ_CD == @boz_cd')['RN_LOSS_1YR'].sum()
            if ogl_1yr_elp_prm_tty_boz == 0: continue
            loss_ratio = 일반_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
            cv = 산업_손해율_변동계수.query('BOZ_CD == @boz_cd')['CV'].values[0]
            slope, a, b, top, bottom = 일반_특약수수료_가공.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR == @tty_yr')[['CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'TOP_CMSN_RT', 'LWT_CMSN_RT']].values[0]
            ogl_risk, rn_risk = get_ret_risk_rate_by_loss_dist(
                ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty, 
                ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz, 
                loss_ratio, cv, slope, a, b, top, bottom
            )
            일반_보유리스크율_손해율분포법.append([
                boz_cd, tty_yr, tty_cd_grp, ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty,
                ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz,
                loss_ratio, loss_ratio*cv, slope, a, b, top, bottom, ogl_risk, rn_risk
            ])
일반_보유리스크율_손해율분포법 = pd.DataFrame(일반_보유리스크율_손해율분포법, columns=['BOZ_CD', 'TTY_YR', 'TTY_CD_GRP', 'OGL_ELP_PRM_TTY', 'RN_ELP_PRM_TTY', 'OGL_LOSS_TTY', 'RN_LOSS_TTY',
                'OGL_1YR_ELP_PRM_TTY_BOZ', 'RN_1YR_ELP_PRM_TTY_BOZ', 'OGL_1YR_LOSS_TTY_BOZ', 'RN_1YR_LOSS_TTY_BOZ', 
                'LOSS_RATIO_MEAN', 'LOSS_RATIO_STD', 'SLOPE', 'A', 'B', 'TOP', 'BOTTOM', 'OGL_RISK', 'RN_RISK'])
일반_보유리스크율_손해율분포법.query('OGL_1YR_ELP_PRM_TTY_BOZ > 0', inplace=True)

In [11]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/일반_보유리스크율_{now}.xlsx') as writer:
    일반_보유리스크율_위험계수적용법.to_excel(writer, '일반_보유리스크율_위험계수적용법', index=False)
    일반_보유리스크율_손해율분포법.to_excel(writer, '일반_보유리스크율_손해율분포법', index=False)